# 1. Package 설치

In [ ]:
%pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone

# 2. Knowledge Base 구성을 위한 데이터 생성

- Chroma를 활용한 2. LangChain과 Chroma를 활용한 RAG 구성과 동일함
- Vector Database만 Pinecone으로 변경

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

# 민법
# document_name = './civil_with_markdown.docx'
# 형법
document_name = './criminal_with_markdown.docx'

loader = Docx2txtLoader(document_name)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
len_list = len(document_list)
len_list

43

In [3]:
document_list[len_list - 10]

Document(metadata={'source': './criminal_with_markdown.docx'}, page_content='② 제287조부터 제289조까지의 죄를 범하여 약취, 유인, 매매 또는 이송된 사람을 상해에 이르게 한 때에는 2년 이상 20년 이하의 징역에 처한다.\n\n[전문개정 2013. 4. 5.]\n\n\n\n제291조(약취, 유인, 매매, 이송 등 살인ㆍ치사) ① 제287조부터 제289조까지의 죄를 범하여 약취, 유인, 매매 또는 이송된 사람을 살해한 때에는 사형, 무기 또는 7년 이상의 징역에 처한다.\n\n② 제287조부터 제289조까지의 죄를 범하여 약취, 유인, 매매 또는 이송된 사람을 사망에 이르게 한 때에는 무기 또는 5년 이상의 징역에 처한다.\n\n[전문개정 2013. 4. 5.]\n\n\n\n제292조(약취, 유인, 매매, 이송된 사람의 수수ㆍ은닉 등) ① 제287조부터 제289조까지의 죄로 약취, 유인, 매매 또는 이송된 사람을 수수(授受) 또는 은닉한 사람은 7년 이하의 징역에 처한다.\n\n② 제287조부터 제289조까지의 죄를 범할 목적으로 사람을 모집, 운송, 전달한 사람도 제1항과 동일한 형으로 처벌한다.\n\n[전문개정 2013. 4. 5.]\n\n\n\n제293조 삭제 <2013. 4. 5.>\n\n\n\n제294조(미수범) 제287조부터 제289조까지, 제290조제1항, 제291조제1항과 제292조제1항의 미수범은 처벌한다.\n\n[전문개정 2013. 4. 5.]\n\n\n\n제295조(벌금의 병과) 제288조부터 제291조까지, 제292조제1항의 죄와 그 미수범에 대하여는 5천만원 이하의 벌금을 병과할 수 있다.\n\n[전문개정 2013. 4. 5.]\n\n\n\n제295조의2(형의 감경) 제287조부터 제290조까지, 제292조와 제294조의 죄를 범한 사람이 약취, 유인, 매매 또는 이송된 사람을 안전한 장소로 풀어준 때에는 그 형을 감경할 수 있다.\n\n[전문개정 2013. 4. 5.]\

In [4]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [5]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

# 민법
# index_name = 'civil-markdown-index'
# 형법
index_name = 'criminal-markdown-index'

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

/Users/gonnim/.pyenv/versions/3.12.4/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
# 민법
# query = '부동산 매매 계약에서의 소유 이전은 어떻게 이루어지나요?'
# 형법
query = '소유권 이전 과정에서 허위 서류를 제출한 경우, 이는 어떤 형법적 처벌을 받을 수 있나요?'

# 3. 답변 생성을 위한 Retrieval

- RetrievalQA에 전달하기 위해 retriever 생성
- search_kwargs의 k값을 변경해서 가져올 문서의 갯수를 지정할 수 있음
- .invoke()를 호출해서 어떤 문서를 가져오는지 확인 가능

In [8]:
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(metadata={'source': './civil_with_markdown.docx'}, page_content='②상대부담있는 증여에 대하여는 증여자는 그 부담의 한도에서 매도인과 같은 담보의 책임이 있다.\n\n\n\n제560조(정기증여와 사망으로 인한 실효) 정기의 급여를 목적으로 한 증여는 증여자 또는 수증자의 사망으로 인하여 그 효력을 잃는다.\n\n\n\n제561조(부담부증여) 상대부담있는 증여에 대하여는 본절의 규정외에 쌍무계약에 관한 규정을 적용한다.\n\n\n\n제562조(사인증여) 증여자의 사망으로 인하여 효력이 생길 증여에는 유증에 관한 규정을 준용한다.\n\n\n\n제3절 매매\n\n\n\n제1관 총칙\n\n\n\n제563조(매매의 의의) 매매는 당사자 일방이 재산권을 상대방에게 이전할 것을 약정하고 상대방이 그 대금을 지급할 것을 약정함으로써 그 효력이 생긴다.\n\n\n\n제564조(매매의 일방예약) ①매매의 일방예약은 상대방이 매매를 완결할 의사를 표시하는 때에 매매의 효력이 생긴다.\n\n②전항의 의사표시의 기간을 정하지 아니한 때에는 예약자는 상당한 기간을 정하여 매매완결여부의 확답을 상대방에게 최고할 수 있다.\n\n③예약자가 전항의 기간내에 확답을 받지 못한 때에는 예약은 그 효력을 잃는다.\n\n\n\n제565조(해약금) ①매매의 당사자 일방이 계약당시에 금전 기타 물건을 계약금, 보증금등의 명목으로 상대방에게 교부한 때에는 당사자간에 다른 약정이 없는 한 당사자의 일방이 이행에 착수할 때까지 교부자는 이를 포기하고 수령자는 그 배액을 상환하여 매매계약을 해제할 수 있다.\n\n②제551조의 규정은 전항의 경우에 이를 적용하지 아니한다.\n\n\n\n제566조(매매계약의 비용의 부담) 매매계약에 관한 비용은 당사자 쌍방이 균분하여 부담한다.\n\n\n\n제567조(유상계약에의 준용) 본절의 규정은 매매 이외의 유상계약에 준용한다. 그러나 그 계약의 성질이 이를 허용하지 아니하는 때에는 그러하지 아니하다.\

# 4. Augmentation을 위한 Prompt 활용

- Retrieval 된 데이터는 LangChain에서 제공하는 프롬프트("rlm/rag-prompt") 사용

In [9]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

# 5. 답변 생성

- RetrievalQA를 통해 LLM에 전달
    - RetrievalQA는 create_retrieval_chain으로 대체됨
    - 실제 ChatBot 구현 시 create_retrieval_chain으로 변경하는 과정을 볼 수 있음

In [11]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [12]:
ai_message = qa_chain.invoke({"query": query})

In [13]:
ai_message

{'query': '부동산 매매 계약에서의 소유 이전은 어떻게 이루어지나요?',
 'result': '부동산 매매 계약에서 소유 이전은 매도인이 매수인에게 매매의 목적이 된 권리를 이전하고, 매수인은 매도인에게 대금을 지급함으로써 이루어집니다. 이 쌍방의무는 특별한 약정이나 관습이 없으면 동시에 이행되어야 합니다. (제568조)'}